# 課題3：タイタニック号乗客の生存状況の分類モデル作成

本課題では、`titanic` というデータセットを使います。これは、1912年に発生したタイタニック号の沈没事故における乗客の生存状況に関するデータセットです。元々は、[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/)で掲載されたデータと言われており、このデータセットを組み込んだPythonのライブラリも複数あります。

今回は、`seaborn` のライブラリに組み込まれた `titanic` のデータセットを使います。各セルに入っているコメントの下に、実行するコードを記入してください。わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [271]:
# 必要なライブラリのimport（変更しないでください）
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

# DataFrameですべての列を表示する設定（変更しないでください）
pd.options.display.max_columns = None

## 2. データの読み込み

seabornに添付のデータセットから「titanic」を読み込み、内容を確認します。

In [272]:
# seabornからtitanicのデータセットを読み込む（変更しないでください）
dataset = sns.load_dataset("titanic")

`sns.load_dataset()` で読み込んだデータは、pandasのDataFrameになっています。

In [273]:
# datasetの先頭5件を確認
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 使用する列の指定

今回は `survived, pclass, sex, age, sibsp, parch, fare, embarked` の列を使用します。

#### 参考:各列の説明

- `survived`: 生存区分（0:死亡, 1:生存）
- `pclass`: チケットクラス
- `sex`: 性別（male:男性, female:女性）
- `age`: 年齢
- `sibsp`: 同乗している兄弟や配偶者の数
- `parch`: 同乗している親や子供の数
- `fare`: 料金
- `embarked`: 乗船した港（頭文字）
- `class`: 客室クラス
- `who`: 性別（man:男性, woman:女性）
- `adult_male`: 成人男性ならTrue
- `deck`: 事故の際にどのデッキにいたか
- `embark_town`: 乗船した港名
- `alive`: 生存区分（no:死亡, yes:生存）
- `alone`: 1人で乗船したか

In [274]:
# datasetから「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を取得して
# datasetに代入（上書き）する
dataset = dataset[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

In [275]:
# 改めてdatasetの先頭5件を表示
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## 3. データの前処理

### 要約統計量の表示

In [276]:
# 要約統計量を表示
dataset.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 欠損値の確認と補完

In [277]:
# 各列の欠損値の数を確認
dataset.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

ageの欠損値は平均値で補完します。

In [278]:
# ageの欠損値を、ageの平均値で補完する
dataset['age'] = dataset['age'].fillna(dataset['age'].mean())

embarkedの欠損値は、もっとも乗船者数の多い港で補完します。

その方法はいくつかありますが、ここではその1つとして、DataFrameの特定の1列（Series）が持つ `value_counts()` メソッドを紹介します。このメソッドを実行すると、その列が持つ値ごとのデータ数がわかります。

参考：[pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

In [279]:
# 乗船者数の多い港を value_counts メソッドで確認
dataset['embarked'].value_counts()

embarked
S    644
C    168
Q     77
Name: count, dtype: int64

`values_count()` の結果を見て、もっとも乗船者数の多い港の文字で欠損値を埋めるようにします。

In [280]:
# embarkedの欠損値をもっとも乗船者数の多い港にて補完
dataset['embarked'] = dataset['embarked'].fillna('s')

上記の処理により、欠損値がなくなったかを確認しましょう。

In [281]:
# 欠損値の数を確認し、補完後の欠損値が0であることを確認
dataset.isnull().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

### ダミー変数への変換

sexとembarkedをダミー変数に変換します。

In [282]:
# datasetのsexとembarkedをダミー変数に変換してdataset2に代入する
dataset2 = pd.get_dummies(data=dataset, columns=['sex', 'embarked'])

In [283]:
# dataset2のデータの最初の5行を表示
dataset2.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 4. 目的変数と説明変数の選択

ここでは、以下の列を使用します。

- 目的変数: `survived`
- 説明変数: それ以外

dataset2より目的変数と説明変数に該当する列を取得してnumpy配列に変換し、変数YとXに格納します。列の除外には、DataFrameの `drop` を使います。`データフレーム.drop(columns=除外したい列名)` です。

In [284]:
# Y:目的変数に該当する列
Y = np.array(dataset2['survived'])

# X:説明変数に該当する列。dataset2からsurvivedを除外
X = np.array(dataset2.drop(columns=['survived']))

In [285]:
# YとXの形状を確認
print("Y=", Y.shape, ", X=", X.shape)

Y= (891,) , X= (891, 11)


## 5. データの分割

この課題ではホールドアウト法でデータを分割します。

In [286]:
# X と Y を 機械学習用データとテストデータに7:3に分ける(X_train, X_test, Y_train, Y_test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)


In [287]:
# 機械学習用データを、学習データと検証データに7:3に分ける(X_train, X_valid, Y_train, Y_valid)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

In [288]:
# 形状を確認:X_train, X_valid, X_test, Y_train, Y_valid, Y_test
print("Y_train=", Y_train.shape, ", X_train=", X_train.shape)
print("Y_valid=", Y_valid.shape, ", X_valid=", X_valid.shape)
print("Y_test=", Y_test.shape, ", X_test=", X_test.shape)

Y_train= (436,) , X_train= (436, 11)
Y_valid= (187,) , X_valid= (187, 11)
Y_test= (268,) , X_test= (268, 11)


## 6. モデルの選択

ロジスティック回帰と決定木、ランダムフォレスト、SVMの4つのモデルを作成して比較します。

In [289]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

モデルの評価（性能の比較）には、F1値を使ってください。以下には1つだけセルを用意していますが、モデルを4つ作って比較する処理のためにセルを増やしてもかまいません。

In [290]:
# 4つのモデルを作成し、それぞれのF1値を出力する(ロジスティック回帰)
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train, Y_train)
Y_pred_lr = lr.predict(X_valid) # X_valid/Y_validを使っていると仮定
f1_lr = f1_score(Y_valid, Y_pred_lr)
f1_score(Y_valid, Y_pred)

0.43859649122807015

In [291]:
# 4つのモデルを作成し、それぞれのF1値を出力する(決定木)
tree_model = DecisionTreeClassifier(criterion='entropy')
tree_model.fit(X_train, Y_train)
Y_pred = tree_model.predict(X_valid)

f1_score(Y_valid, Y_pred)

0.7027027027027027

In [292]:
# 4つのモデルを作成し、それぞれのF1値を出力する(ランダムフォレスト)
randomforest_model = RandomForestClassifier(n_estimators=100)
randomforest_model.fit(X_train, Y_train)
Y_pred = randomforest_model.predict(X_valid)

f1_score(Y_valid, Y_pred)

0.7671232876712328

In [293]:
# 4つのモデルを作成し、それぞれのF1値を出力する(SVM)
svm_model = SVC()
svm_model.fit(X_train, Y_train)
Y_pred = svm_model.predict(X_valid)

f1_score(Y_valid, Y_pred)

0.43859649122807015

## 7. パラメータのチューニング

GridSearchCVを使い、性能の良かったランダムフォレストのパラメータのチューニングを行ないます。パラメータの候補については、レッスン本編を参考にしてください。

In [294]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.model_selection import GridSearchCV

In [295]:
# 性能の良かったモデルを作成
# モデル:ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)

# パラメータ:n_estimators:木の数、max_depth:木の深さ
params = {'n_estimators': [10, 50, 100], 'max_depth': [5, 10, 50]}

# グリッドサーチの設定
# cv:交差検証の回数, scoring:目的変数
gscv = GridSearchCV(model, param_grid=params, cv=3, scoring='neg_mean_squared_error')

In [296]:
# パラメータの指定


In [297]:
# グリッドサーチのオブジェクトを作成


In [298]:
# データの分割:機械学習用データを学習と検証に分けるのはクロスバリデーションで行ってくれる
# （Xg_train, Xg_test, Yg_train, Yg_test）


In [299]:
# グリッドサーチを実行する


In [300]:
# 最適なパラメータを表示


ここで得たパラメータをもとに、モデルを再度作成します。

In [301]:
# 最適なパラメータによるモデルの作成


# モデルの学習


# モデルの予測


In [302]:
# F1値の出力


## 8. テストデータによる汎化性能の確認

最後にテストデータでモデルの汎化性能を確認しましょう。

In [303]:
# テストデータを使って予測を行いF1値を算出
